# Here i make the same feature engingeering as in dataset generation, but instead of euclidean distance i take walking distance calculated by Krzysztof

In [46]:
import pandas as pd
import h3
traffic_data = pd.read_parquet("../for_participants/data_parquet/traffic_train.parquet")
pois_df = pd.read_parquet("../for_participants/data_parquet/pois.parquet")

In [47]:
wd = pd.read_parquet("../krzysztof/pois_walking_distance.parquet")

In [48]:
df_left = pd.DataFrame(wd["station_h3"].unique())
df_left.columns = ["station_h3"]

In [49]:
results = []
for dist in [0.2, 0.4, 0.6, 0.8, 1]:
    wd0 = wd[wd.walking_distance_from_station_km < dist]
    df0 = df_left.merge(wd0.groupby("station_h3")[['accommodation', 'active_life', 'arts_and_entertainment',
       'attractions_and_activities', 'automotive', 'beauty_and_spa',
       'business_to_business', 'eat_and_drink', 'education',
       'financial_service', 'health_and_medical', 'home_service', 'mass_media',
       'pets', 'private_establishments_and_corporates',
       'professional_services', 'public_service_and_government', 'real_estate',
       'religious_organization', 'retail', 'structure_and_geography', 'travel']].sum(), left_on = "station_h3", right_index = True, how = "left")
    new_columns = {
        col: col + "_wd" + str(dist) if col != "station_h3" else col
        for col in df0.columns
    }
    df0 = df0.rename(columns = new_columns)
    results.append(df0)

In [50]:
df_wd = pd.concat(results, axis =1)

In [51]:
df_wd = df_wd.fillna(0)

In [52]:
df_wd = df_wd.loc[:, ~df_wd.columns.duplicated()]

In [53]:
df_wd.to_parquet("dfs_processed/df_pois_agg_wd.parquet")